## 1. Data Loading and Setup {#data-loading}

Let's start by importing necessary libraries and loading our datasets.


In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")


Libraries imported successfully!
Pandas version: 2.2.3
Numpy version: 1.24.3
Matplotlib version: 3.10.3
Seaborn version: 0.13.2


In [8]:
# Function to load and combine all disruption datasets
def load_all_disruption_data(datasets_path='datasets/'):
    """
    Load all CSV files from the datasets directory and combine them into a single DataFrame.
    
    Returns:
        pd.DataFrame: Combined dataset with all years
    """
    datasets_dir = Path(datasets_path)
    csv_files = sorted(datasets_dir.glob('disruptions-*.csv'))
    
    print(f"Found {len(csv_files)} CSV files:")
    for file in csv_files:
        print(f"  - {file.name}")
    
    all_data = []
    total_rows = 0
    
    for file in csv_files:
        print(f"\nLoading {file.name}...")
        try:
            df = pd.read_csv(file)
            
            # Add metadata columns
            year = file.name.split('-')[1].split('.')[0]
            df['year'] = int(year)
            df['source_file'] = file.name
            
            print(f"  Loaded {len(df):,} rows")
            total_rows += len(df)
            all_data.append(df)
            
        except Exception as e:
            print(f"  Error loading {file.name}: {e}")
    
    # Combine all dataframes
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n✅ Successfully combined {len(all_data)} files")
        print(f"📊 Total dataset: {len(combined_df):,} rows, {len(combined_df.columns)} columns")
        return combined_df
    else:
        print("❌ No data loaded!")
        return None

# Load the data
print("🚂 Loading Dutch Train Disruption Data...\n")
df = load_all_disruption_data()

if df is not None:
    print(f"\n🎯 Data loaded successfully!")
    print(f"Date range: {df['year'].min()} - {df['year'].max()}")
else:
    print("⚠️ Failed to load data!")


🚂 Loading Dutch Train Disruption Data...

Found 14 CSV files:
  - disruptions-2011.csv
  - disruptions-2012.csv
  - disruptions-2013.csv
  - disruptions-2014.csv
  - disruptions-2015.csv
  - disruptions-2016.csv
  - disruptions-2017.csv
  - disruptions-2018.csv
  - disruptions-2019.csv
  - disruptions-2020.csv
  - disruptions-2021.csv
  - disruptions-2022.csv
  - disruptions-2023.csv
  - disruptions-2024.csv

Loading disruptions-2011.csv...
  Loaded 1,846 rows

Loading disruptions-2012.csv...
  Loaded 2,074 rows

Loading disruptions-2013.csv...
  Loaded 2,312 rows

Loading disruptions-2014.csv...
  Loaded 2,484 rows

Loading disruptions-2015.csv...
  Loaded 2,947 rows

Loading disruptions-2016.csv...
  Loaded 3,031 rows

Loading disruptions-2017.csv...
  Loaded 4,085 rows

Loading disruptions-2018.csv...
  Loaded 5,190 rows

Loading disruptions-2019.csv...
  Loaded 5,940 rows

Loading disruptions-2020.csv...
  Loaded 4,450 rows

Loading disruptions-2021.csv...
  Loaded 4,874 rows

Load

In [9]:
# Data preprocessing and cleaning
def preprocess_data(df):
    """
    Clean and preprocess the disruption data.
    
    Args:
        df (pd.DataFrame): Raw disruption data
        
    Returns:
        pd.DataFrame: Cleaned and processed data
    """
    print("🔧 Preprocessing data...")
    
    # Make a copy to avoid modifying original
    df_clean = df.copy()
    
    # Convert datetime columns
    df_clean['start_time'] = pd.to_datetime(df_clean['start_time'])
    df_clean['end_time'] = pd.to_datetime(df_clean['end_time'])
    
    # Extract time-based features
    df_clean['start_date'] = df_clean['start_time'].dt.date
    df_clean['start_hour'] = df_clean['start_time'].dt.hour
    df_clean['start_month'] = df_clean['start_time'].dt.month
    df_clean['start_weekday'] = df_clean['start_time'].dt.dayofweek  # 0=Monday
    df_clean['start_weekday_name'] = df_clean['start_time'].dt.day_name()
    df_clean['start_month_name'] = df_clean['start_time'].dt.month_name()
    
    # Add season classification
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Autumn'
    
    df_clean['season'] = df_clean['start_month'].apply(get_season)
    
    # Add time period classification
    def get_time_period(hour):
        if 5 <= hour < 9:
            return 'Morning Rush'
        elif 9 <= hour < 17:
            return 'Day Time'
        elif 17 <= hour < 20:
            return 'Evening Rush'
        elif 20 <= hour < 24:
            return 'Evening'
        else:
            return 'Night'
    
    df_clean['time_period'] = df_clean['start_hour'].apply(get_time_period)
    
    # Count affected stations
    df_clean['station_count'] = df_clean['rdt_station_names'].fillna('').apply(
        lambda x: len([s.strip() for s in x.split(',') if s.strip()])
    )
    
    # Boolean for weekend
    df_clean['is_weekend'] = df_clean['start_weekday'].isin([5, 6])
    
    # Duration categories
    def categorize_duration(minutes):
        if minutes <= 30:
            return 'Short (≤30 min)'
        elif minutes <= 120:
            return 'Medium (30-120 min)'
        elif minutes <= 360:
            return 'Long (2-6 hours)'
        else:
            return 'Very Long (>6 hours)'
    
    df_clean['duration_category'] = df_clean['duration_minutes'].apply(categorize_duration)
    
    print(f"✅ Preprocessing complete!")
    print(f"📊 Added {len(df_clean.columns) - len(df.columns)} new features")
    
    return df_clean

# Apply preprocessing
if df is not None:
    df_processed = preprocess_data(df)
    print(f"\n📈 Final dataset shape: {df_processed.shape}")
else:
    print("⚠️ No data to preprocess!")


🔧 Preprocessing data...
✅ Preprocessing complete!
📊 Added 11 new features

📈 Final dataset shape: (55864, 27)


## 2. Dataset Overview {#dataset-overview}

Let's explore the basic characteristics of our combined dataset.


In [10]:
# Dataset basic information
if df_processed is not None:
    print("📋 DATASET OVERVIEW")
    print("=" * 50)
    
    print(f"📊 Total Disruptions: {len(df_processed):,}")
    print(f"📅 Date Range: {df_processed['start_time'].min()} to {df_processed['start_time'].max()}")
    print(f"🗓️ Years Covered: {df_processed['year'].nunique()} years ({df_processed['year'].min()}-{df_processed['year'].max()})")
    print(f"📍 Unique Routes: {df_processed['rdt_lines'].nunique():,}")
    print(f"🚉 Unique Station Names: {df_processed['rdt_station_names'].nunique():,}")
    
    print(f"\n⏱️ DURATION STATISTICS")
    print("=" * 50)
    print(f"Average Duration: {df_processed['duration_minutes'].mean():.1f} minutes")
    print(f"Median Duration: {df_processed['duration_minutes'].median():.1f} minutes")
    print(f"Shortest Disruption: {df_processed['duration_minutes'].min():.1f} minutes")
    print(f"Longest Disruption: {df_processed['duration_minutes'].max():.1f} minutes")
    
    print(f"\n📈 YEARLY BREAKDOWN")
    print("=" * 50)
    yearly_counts = df_processed['year'].value_counts().sort_index()
    for year, count in yearly_counts.items():
        print(f"{year}: {count:,} disruptions")
    
    print(f"\n🎯 CAUSE GROUPS")
    print("=" * 50)
    cause_counts = df_processed['cause_group'].value_counts()
    for cause, count in cause_counts.items():
        percentage = (count / len(df_processed)) * 100
        print(f"{cause}: {count:,} ({percentage:.1f}%)")
        
    print(f"\n📋 DATASET INFO")
    print("=" * 50)
    print(df_processed.info())


📋 DATASET OVERVIEW
📊 Total Disruptions: 55,864
📅 Date Range: 2011-01-03 23:22:50 to 2024-12-31 16:09:32
🗓️ Years Covered: 14 years (2011-2024)
📍 Unique Routes: 661
🚉 Unique Station Names: 1,975

⏱️ DURATION STATISTICS
Average Duration: 159.2 minutes
Median Duration: 62.0 minutes
Shortest Disruption: 0.0 minutes
Longest Disruption: 64927.0 minutes

📈 YEARLY BREAKDOWN
2011: 1,846 disruptions
2012: 2,074 disruptions
2013: 2,312 disruptions
2014: 2,484 disruptions
2015: 2,947 disruptions
2016: 3,031 disruptions
2017: 4,085 disruptions
2018: 5,190 disruptions
2019: 5,940 disruptions
2020: 4,450 disruptions
2021: 4,874 disruptions
2022: 5,499 disruptions
2023: 5,168 disruptions
2024: 5,964 disruptions

🎯 CAUSE GROUPS
rolling stock: 20,111 (36.0%)
infrastructure: 15,527 (27.8%)
accidents: 6,190 (11.1%)
external: 5,965 (10.7%)
engineering work: 2,716 (4.9%)
logistical: 2,403 (4.3%)
unknown: 1,164 (2.1%)
staff: 991 (1.8%)
weather: 795 (1.4%)

📋 DATASET INFO
<class 'pandas.core.frame.DataFrame'>